In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('And18.phot2', delim_whitespace=True, header=None)


In [4]:
df = df[[0,2,3,16,29]]
df.columns=['chip','x','y','V','I']
df['V-I']=df['V']-df['I']

In [5]:
df['y'] = df.apply(lambda x: x.y+2000 if x.chip>1 else x.y, axis=1)

In [12]:
from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
output_notebook()

s1 = ColumnDataSource(data=df)
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6, size=0.1)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=400, plot_height=400, x_range=[-1.5,2], y_range=[27,20],
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6, size=0.1)

s1.callback = CustomJS(args=dict(s2=s2, p1=p1), code="""
        var inds = cb_obj.selected['1d'].indices;
        var d1 = cb_obj.data;
        var d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (i = 0; i < inds.length; i++) {
            d2['x'].push(d1['V-I'][inds[i]])
            d2['y'].push(d1['I'][inds[i]])
        }
        s2.change.emit();
        p1.title="Selected";
        p1.change.emit();
    """)

button = Button(label="Download", button_type="success")
button.callback = CustomJS(args=dict(source=source),
                           code=open(join(dirname(__file__), "download.js")).read())

layout = row(p1, p2)

show(layout)

Loading BokehJS ...

In [20]:
idx = s1.selected.items

In [21]:
idx

<function PropertyValueDict.items>